In [1]:
import os
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import re
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from tensorflow.python.client import device_lib
import random
from tensorflow.keras.models import Model
from keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib
import cv2
import csv

2022-06-17 18:32:57.496886: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16929524498263938339
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10561793600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 338652547942897482
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:5e:00.0, compute capability: 7.5"
]


2022-06-17 18:32:59.932601: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-17 18:32:59.935158: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-17 18:32:59.936373: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-17 18:32:59.975074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:5e:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-06-17 18:32:59.975113: I tensorflow/stream_executor/platform/default/dso_loader.cc

In [3]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/Sequence_prediction/Sequence_to_sequence/h5_models/')
model_name = "UNet_Xception_style_model_24.h5" 
model = load_model(model_name, compile=False)
model.summary()

2022-06-17 18:33:01.253626: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-17 18:33:01.254936: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:5e:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-06-17 18:33:01.254997: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-06-17 18:33:01.255068: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-06-17 18:33:01.255080: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-06-17 18:33:01.255091: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 512, 512 0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 24, 512, 512, 240         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 24, 512, 512, 96          time_distributed[0][0]           
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 24, 512, 512, 5208        batch_normalization[0][0]        
______________________________________________________________________________________________

In [4]:
path_to_csv = '/home/aijjeh/aijjeh_rexio_share/PhD/cmap_flipped_jet256.csv'
cmap = matplotlib.colors.ListedColormap(["blue", "green", "red"], name=path_to_csv, N=None)

In [5]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [6]:
def append_list_as_row(cvs_file_name, list_of_iou, image_num):
    os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical_results_Full_wavefield/csv_files/')
    with open(cvs_file_name, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerows([image_num])
        writer.writerows([list_of_iou])

In [7]:
def plot_res(img, ext, frame):
    ############################################################################################################
    plt.figure(figsize=(1, 1), dpi=512)
    plt.gca().set_axis_off()
    plt.axis('off')
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1.0, wspace=0.0, hspace=0.0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    ############################################################################################################
    plt.imshow(img, cmap=cmap)
    img = np.asarray(img)
    # img = img *255
    # img = img.astype(np.uint8)
    # cv2.imwrite('frame_%d_case_%d.png' % (frame, case), img)
    plt.savefig('frame_%d_case_%d.png' % (frame, case))
    # np.save('frame_%d_case_%d' % (frame, case), img)
    # plt.show()
    plt.close('all')

In [8]:
def calc_IoU(predicted_image, truth_img, frame):
    predicted_image = np.asarray(predicted_image) # 
    predicted_image = predicted_image.astype('float64')
    ret, predicted_image = cv2.threshold(predicted_image, .5, 1.0, cv2.THRESH_BINARY)
    truth_img = np.asarray(truth_img)# 
    truth_img = truth_img.astype('float64')
    ret_, truth_img = cv2.threshold(truth_img, .5, 1.0, cv2.THRESH_BINARY)

    truth_img = truth_img[:,:,0]
    InterSectionArray = cv2.bitwise_and(predicted_image, truth_img)
    UnionArray = cv2.bitwise_or(predicted_image, truth_img)
    I1 = np.count_nonzero(InterSectionArray)
    U = np.count_nonzero(UnionArray)
    IoU = I1 / U
    print('frame_%d_iou : ' % frame,IoU)
    return IoU

In [9]:
class Full_wavefield_frames(keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_img_paths_total_, target_img_paths_, time_stamps, frame_init):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths_total = input_img_paths_total_
        self.target_img_paths = target_img_paths_
        self.time_stamps = time_stamps
        self.frame_num = frame_init
    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size        
        batch_input_img_paths = self.input_img_paths_total[i:i+self.batch_size]      
        batch_target_img_paths = self.target_img_paths[i:i+self.batch_size] 
        
        x = np.zeros((self.batch_size,) + (self.time_stamps,) + self.img_size + (1,), dtype="float32") #  
        
        for batch_num in range(self.batch_size):
            batch_input_img_paths = batch_input_img_paths[batch_num][self.frame_num:self.frame_num+self.time_stamps] 
            for j, path in enumerate(batch_input_img_paths):
                img = load_img(path, target_size=self.img_size, color_mode="grayscale")            
                img = np.expand_dims(img,2)
                img = img / 255.0
                x[batch_num][j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")            
            img = np.expand_dims(img, 2)
            img = img / 255.0
            y[j] = img                
        return x, y

In [10]:
target_dir = "/aijjeh_odroid_sensors/aidd/data/raw/num/dataset2_labels_out/"    
########################################################
height =  512 
width =  512 
img_size = (height, width)
num_classes = 2
batch_size = 1
time_stamps = 24 
input_img_paths_total = [None]*1
######################################################## 
# exp_path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/Numerical_results_Full_wavefield/'
exp_path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/filtered/'

In [11]:
for case in [438, 448, 456, 475]:
    
    list_frames = []
    list_iou = []
        
    total_IoU  = 0
    
    cvs_file_name = 'SR_num_input_sameple_case_%d.csv' %case
    
    exp_gt = 'm1_rand_single_delam_%d.png' % case
    
    # input_dir = "/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical_results_Full_wavefield/%d_output/" % case
    # input_dir = "/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical_results_Full_wavefield/median_filter_%d_output/" % case
    input_dir = "/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/%d_output" % case

    
    my_list = os.listdir(input_dir)
    my_list.sort(key=natural_keys)
    print(len(my_list))
    
    
    input_img_paths = sorted(
        [
            os.path.join(input_dir, fname) 
            for fname in os.listdir(input_dir)    
            if fname.endswith(".png")
        ])
    input_img_paths.sort(key=natural_keys)
    input_img_paths_total[0]= input_img_paths
    print((input_img_paths_total))
    
    
    target_img_paths = sorted(
        [
            os.path.join(target_dir, fname)
            for fname in os.listdir(target_dir)
            if fname.endswith(exp_gt) and not fname.startswith(".")
        ]
    )
    target_img_paths.sort(key=natural_keys)
    print((target_img_paths))
    
    
    test_input_img_paths = input_img_paths_total[:]
    test_target_img_paths = target_img_paths[:]            
            
    
    os.chdir(exp_path)
    exp_dir = 'SR_Num_input_Prediction_frames_case_%d' % case
    if not os.path.exists(exp_dir):
        os.mkdir(exp_dir)
    os.chdir(exp_dir)
    
        
    for frame_init in range(25, len(my_list)-24):
        val_gen = Full_wavefield_frames(batch_size, img_size, input_img_paths_total, target_img_paths, time_stamps, frame_init)
        val_preds = model.predict(val_gen)
        
        # Display ground-truth target mask
        
        img_GT_ = PIL.ImageOps.autocontrast(load_img(test_target_img_paths[0], target_size= img_size))
        
        # Display mask predicted by our model
        
        mask = val_preds[0]
        img_pred_ = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask)) # 
        plot_res(keras.preprocessing.image.array_to_img(mask), 'predicted_output', frame_init)
        iou = calc_IoU(img_pred_, img_GT_, frame_init)       
        total_IoU = total_IoU + iou
        list_frames.append(frame_init)
        list_iou.append(iou)
        
    
    # append_list_as_row(cvs_file_name, list_iou, list_frames)

512
[['/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_1_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_2_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_3_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_4_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_5_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_6_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_7_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_8_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_results/num/filtered/438_output/frame_9_SR_num_pred.png', '/aijjeh_odroid_laser/aidd/Compressive_sensing_re

2022-06-17 18:33:02.996529: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-17 18:33:03.016249: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz
2022-06-17 18:33:04.252135: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-06-17 18:33:06.683176: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-06-17 18:33:07.211576: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


frame_25_iou :  0.0
frame_26_iou :  0.0
frame_27_iou :  0.0
frame_28_iou :  0.0
frame_29_iou :  0.0
frame_30_iou :  0.0
frame_31_iou :  0.0
frame_32_iou :  0.0
frame_33_iou :  0.0
frame_34_iou :  0.0
frame_35_iou :  0.0
frame_36_iou :  0.0023940627244433805
frame_37_iou :  0.0005952735281862016
frame_38_iou :  0.0005676013168350551
frame_39_iou :  0.0005446623093681918
frame_40_iou :  0.0005509641873278236
frame_41_iou :  0.0005259006047856955
frame_42_iou :  0.0005214308061320263
frame_43_iou :  0.0005065086359722433
frame_44_iou :  0.000478080030597122
frame_45_iou :  0.0004657444925713753
frame_46_iou :  0.0004601297565913588
frame_47_iou :  0.00045014629754670267
frame_48_iou :  0.00043950248318903
frame_49_iou :  0.0004359577992850292
frame_50_iou :  0.00040981926970206137
frame_51_iou :  0.00039022867400296575
frame_52_iou :  0.0003780146669690784
frame_53_iou :  0.00037221767289510904
frame_54_iou :  0.0003778575477045154
frame_55_iou :  0.0003635173943073176
frame_56_iou :  0.0

frame_312_iou :  0.0
frame_313_iou :  0.0
frame_314_iou :  0.0
frame_315_iou :  0.0
frame_316_iou :  0.0
frame_317_iou :  0.0
frame_318_iou :  0.0
frame_319_iou :  0.0
frame_320_iou :  0.0
frame_321_iou :  0.0
frame_322_iou :  0.0
frame_323_iou :  0.0
frame_324_iou :  0.0
frame_325_iou :  0.001430615164520744
frame_326_iou :  0.0013623978201634877
frame_327_iou :  0.0011210762331838565
frame_328_iou :  0.0
frame_329_iou :  0.0
frame_330_iou :  0.0022172949002217295
frame_331_iou :  0.0026431718061674008
frame_332_iou :  0.0072992700729927005
frame_333_iou :  0.009243697478991597
frame_334_iou :  0.0057553956834532375
frame_335_iou :  0.002512562814070352
frame_336_iou :  0.0006207324643078833
frame_337_iou :  0.0
frame_338_iou :  0.003538570417551309
frame_339_iou :  0.008270158511371467
frame_340_iou :  0.003893575600259572
frame_341_iou :  0.0018820577164366374
frame_342_iou :  0.0
frame_343_iou :  0.0
frame_344_iou :  0.0
frame_345_iou :  0.0
frame_346_iou :  0.0
frame_347_iou :  0.

frame_25_iou :  0.0
frame_26_iou :  0.0
frame_27_iou :  0.0
frame_28_iou :  0.0
frame_29_iou :  0.0
frame_30_iou :  0.0
frame_31_iou :  0.0
frame_32_iou :  0.0
frame_33_iou :  0.0
frame_34_iou :  0.0
frame_35_iou :  0.0
frame_36_iou :  0.0
frame_37_iou :  0.0022414733143514875
frame_38_iou :  0.0020558982052564317
frame_39_iou :  0.00194001677852349
frame_40_iou :  0.0019494204425711274
frame_41_iou :  0.0018662362554221728
frame_42_iou :  0.0018089371272122811
frame_43_iou :  0.0017767106842737094
frame_44_iou :  0.0017090858700170909
frame_45_iou :  0.0016557032263838547
frame_46_iou :  0.00164407909353477
frame_47_iou :  0.0016217400832785449
frame_48_iou :  0.0015971682638349304
frame_49_iou :  0.0015660712774062473
frame_50_iou :  0.001471816699152711
frame_51_iou :  0.001383022464770306
frame_52_iou :  0.0013506607286267066
frame_53_iou :  0.0013454056216137594
frame_54_iou :  0.0013565536205316223
frame_55_iou :  0.001314060446780552
frame_56_iou :  0.0013025417165387595
frame_5

frame_342_iou :  0.0
frame_343_iou :  0.0
frame_344_iou :  0.0
frame_345_iou :  0.0
frame_346_iou :  0.0
frame_347_iou :  0.0006218905472636816
frame_348_iou :  0.0005961251862891207
frame_349_iou :  0.0
frame_350_iou :  0.0
frame_351_iou :  0.0
frame_352_iou :  0.0
frame_353_iou :  0.0
frame_354_iou :  0.0
frame_355_iou :  0.0
frame_356_iou :  0.0
frame_357_iou :  0.0
frame_358_iou :  0.0
frame_359_iou :  0.0
frame_360_iou :  0.0
frame_361_iou :  0.0
frame_362_iou :  0.0
frame_363_iou :  0.0
frame_364_iou :  0.0
frame_365_iou :  0.0
frame_366_iou :  0.0
frame_367_iou :  0.0
frame_368_iou :  0.0
frame_369_iou :  0.0
frame_370_iou :  0.0
frame_371_iou :  0.0
frame_372_iou :  0.0
frame_373_iou :  0.0
frame_374_iou :  0.0
frame_375_iou :  0.0
frame_376_iou :  0.0
frame_377_iou :  0.0
frame_378_iou :  0.0
frame_379_iou :  0.0
frame_380_iou :  0.0
frame_381_iou :  0.0
frame_382_iou :  0.0
frame_383_iou :  0.0
frame_384_iou :  0.0
frame_385_iou :  0.0
frame_386_iou :  0.0
frame_387_iou :  0.

frame_25_iou :  0.0
frame_26_iou :  0.0
frame_27_iou :  0.0
frame_28_iou :  0.0
frame_29_iou :  0.0
frame_30_iou :  0.0
frame_31_iou :  0.0
frame_32_iou :  0.0
frame_33_iou :  0.0
frame_34_iou :  0.0
frame_35_iou :  0.0
frame_36_iou :  0.00047904191616766467
frame_37_iou :  0.0013481038626106325
frame_38_iou :  0.0012553214714550814
frame_39_iou :  0.0012552301255230125
frame_40_iou :  0.0012295520153961295
frame_41_iou :  0.001170648645772362
frame_42_iou :  0.0011800042909246942
frame_43_iou :  0.0011209619891980026
frame_44_iou :  0.0010895310279488393
frame_45_iou :  0.0009986835534976622
frame_46_iou :  0.001121327651939897
frame_47_iou :  0.0011588518452487074
frame_48_iou :  0.0010418927718688952
frame_49_iou :  0.0010520114458845312
frame_50_iou :  0.0009976853699417351
frame_51_iou :  0.0009504980609839555
frame_52_iou :  0.0009399909760866295
frame_53_iou :  0.0008600703013985491
frame_54_iou :  0.0008626509639186857
frame_55_iou :  0.0008512528220881602
frame_56_iou :  0.000

frame_314_iou :  0.0
frame_315_iou :  0.0
frame_316_iou :  0.0
frame_317_iou :  0.0
frame_318_iou :  0.0
frame_319_iou :  0.0
frame_320_iou :  0.0
frame_321_iou :  0.0
frame_322_iou :  0.0
frame_323_iou :  0.0
frame_324_iou :  0.0
frame_325_iou :  0.0
frame_326_iou :  0.0
frame_327_iou :  0.0
frame_328_iou :  0.0
frame_329_iou :  0.0
frame_330_iou :  0.0
frame_331_iou :  0.0
frame_332_iou :  0.0
frame_333_iou :  0.0
frame_334_iou :  0.0
frame_335_iou :  0.0
frame_336_iou :  0.0
frame_337_iou :  0.0
frame_338_iou :  0.0
frame_339_iou :  0.0
frame_340_iou :  0.0
frame_341_iou :  0.0
frame_342_iou :  0.0
frame_343_iou :  0.0
frame_344_iou :  0.0
frame_345_iou :  0.0
frame_346_iou :  0.0
frame_347_iou :  0.0
frame_348_iou :  0.0
frame_349_iou :  0.0
frame_350_iou :  0.0
frame_351_iou :  0.0
frame_352_iou :  0.0
frame_353_iou :  0.0
frame_354_iou :  0.0
frame_355_iou :  0.0
frame_356_iou :  0.0
frame_357_iou :  0.0
frame_358_iou :  0.0
frame_359_iou :  0.0
frame_360_iou :  0.0
frame_361_iou

['/aijjeh_odroid_sensors/aidd/data/raw/num/dataset2_labels_out/m1_rand_single_delam_475.png']
frame_25_iou :  0.0
frame_26_iou :  0.0
frame_27_iou :  0.0
frame_28_iou :  0.0
frame_29_iou :  0.0
frame_30_iou :  0.0
frame_31_iou :  0.0
frame_32_iou :  0.0
frame_33_iou :  0.0
frame_34_iou :  0.0
frame_35_iou :  0.0
frame_36_iou :  0.0
frame_37_iou :  0.0
frame_38_iou :  0.0
frame_39_iou :  0.0
frame_40_iou :  0.0
frame_41_iou :  0.0
frame_42_iou :  0.0
frame_43_iou :  0.0
frame_44_iou :  0.0
frame_45_iou :  0.0
frame_46_iou :  0.0
frame_47_iou :  0.0
frame_48_iou :  0.0
frame_49_iou :  0.0
frame_50_iou :  0.0
frame_51_iou :  0.0
frame_52_iou :  0.0
frame_53_iou :  0.0
frame_54_iou :  0.0
frame_55_iou :  0.0
frame_56_iou :  0.0
frame_57_iou :  0.0
frame_58_iou :  0.0
frame_59_iou :  0.0
frame_60_iou :  0.0
frame_61_iou :  0.0
frame_62_iou :  0.0
frame_63_iou :  0.0
frame_64_iou :  0.0
frame_65_iou :  0.0
frame_66_iou :  0.0
frame_67_iou :  0.0
frame_68_iou :  0.0
frame_69_iou :  0.0
frame_

frame_394_iou :  0.0
frame_395_iou :  0.0
frame_396_iou :  0.0
frame_397_iou :  0.0
frame_398_iou :  0.0
frame_399_iou :  0.0
frame_400_iou :  0.0
frame_401_iou :  0.0
frame_402_iou :  0.0
frame_403_iou :  0.0
frame_404_iou :  0.0
frame_405_iou :  0.0
frame_406_iou :  0.0
frame_407_iou :  0.0
frame_408_iou :  0.0
frame_409_iou :  0.0
frame_410_iou :  0.0
frame_411_iou :  0.0
frame_412_iou :  0.0
frame_413_iou :  0.0
frame_414_iou :  0.0
frame_415_iou :  0.0
frame_416_iou :  0.0
frame_417_iou :  0.0
frame_418_iou :  0.0
frame_419_iou :  0.0
frame_420_iou :  0.0
frame_421_iou :  0.0
frame_422_iou :  0.0
frame_423_iou :  0.0
frame_424_iou :  0.0
frame_425_iou :  0.0
frame_426_iou :  0.0
frame_427_iou :  0.0
frame_428_iou :  0.0
frame_429_iou :  0.0
frame_430_iou :  0.0
frame_431_iou :  0.0
frame_432_iou :  0.0
frame_433_iou :  0.0
frame_434_iou :  0.0
frame_435_iou :  0.0
frame_436_iou :  0.0
frame_437_iou :  0.0
frame_438_iou :  0.0
frame_439_iou :  0.0
frame_440_iou :  0.0
frame_441_iou